In [1]:
import pandas as pd
import numpy as np
import sklearn
import lightgbm as lgb

/Users/wataru-masuda/.virtualenvs/python3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train_data = pd.read_csv("./train_data.csv")
train_data.head()
#train_data.describe()


,id,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,y
0,0,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,1,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,2,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,3,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
4,4,50000,1,1,2,37,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0


In [3]:
train_data.columns

Index(['id', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
       'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20',
       'X21', 'X22', 'X23', 'y'],
      dtype='object')

In [4]:
train_data.shape

(27000, 25)

In [5]:
x = np.array(train_data)[:,:-1]
y = np.array(train_data)[:,-1]

In [6]:
num_train = int(27000*0.8)

In [7]:
x_train = x[:num_train,:]
x_train.shape

(21600, 24)

In [8]:
y_train = y[:num_train]

In [9]:
x_test = x[num_train:,:]
y_test = y[num_train:]

In [10]:
lgbm_params = {
    "objective": "binary",
    #"num_class": 1,
}

In [11]:
lgb_train=lgb.Dataset(x_train, y_train)

In [12]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test,reference=lgb_train)

In [13]:
y_test.shape
x_test.shape

(5400, 24)

In [14]:
model = lgb.train(lgbm_params, lgb_train,valid_sets=lgb_eval)

[1]	valid_0's binary_logloss: 0.498064
[2]	valid_0's binary_logloss: 0.482342
[3]	valid_0's binary_logloss: 0.470819
[4]	valid_0's binary_logloss: 0.461483
[5]	valid_0's binary_logloss: 0.453622
[6]	valid_0's binary_logloss: 0.447604
[7]	valid_0's binary_logloss: 0.442228
[8]	valid_0's binary_logloss: 0.438236
[9]	valid_0's binary_logloss: 0.43462
[10]	valid_0's binary_logloss: 0.431408
[11]	valid_0's binary_logloss: 0.428424
[12]	valid_0's binary_logloss: 0.42574
[13]	valid_0's binary_logloss: 0.423707
[14]	valid_0's binary_logloss: 0.422025
[15]	valid_0's binary_logloss: 0.420465
[16]	valid_0's binary_logloss: 0.419431
[17]	valid_0's binary_logloss: 0.41839
[18]	valid_0's binary_logloss: 0.417516
[19]	valid_0's binary_logloss: 0.416303
[20]	valid_0's binary_logloss: 0.415325
[21]	valid_0's binary_logloss: 0.414666
[22]	valid_0's binary_logloss: 0.413986
[23]	valid_0's binary_logloss: 0.413286
[24]	valid_0's binary_logloss: 0.412984
[25]	valid_0's binary_logloss: 0.412472
[26]	valid_0

In [23]:
y_pred = model.predict(x_test, num_iteration=model.best_iteration)
y_pred[0:10]

array([0.05531255, 0.32400186, 0.61489061, 0.02094477, 0.77198682,
       0.79901986, 0.11480823, 0.13693013, 0.064996  , 0.07209126])

In [25]:
y_pred_max = max(y_pred)
y_pred_max

0.9180049081084017

In [18]:
accuracy = sum(y_test == y_pred_max) / len(y_test)

In [19]:
accuracy

0.0

In [26]:
y_test[:10]

array([0, 0, 0, 0, 1, 1, 0, 0, 1, 0])

In [35]:
res = []
for i in range(len(y_pred)):
    if y_pred[i] > 0.5:
        res.append(1)
    else:
        res.append(0)

In [36]:
accuracy = sum(res == y_test) / len(y_test)
accuracy

0.8312962962962963

In [38]:
test_data = pd.read_csv("./test_data.csv")
test_data.shape

(3000, 24)

In [39]:
lgb_test = lgb.Dataset(test_data, reference=lgb_train)

In [40]:
y_pred_test = model.predict(test_data, num_iteration=model.best_iteration)

In [41]:
res_test = []
for i in range(len(y_pred_test)):
    if y_pred_test[i] > 0.5:
        res_test.append(1)
    else:
        res_test.append(0)

In [42]:
res_test

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,


In [43]:
res_test

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,


In [44]:
df = pd.DataFrame()

In [45]:
df["ID"]=test_data["ID"]

In [46]:
df.head()

,ID
0,0
1,1
2,2
3,3
4,4


In [47]:
df["Y"] = res_test

In [48]:
df.head()

,ID,Y
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [52]:
df.to_csv("submission.csv", index=False)

In [53]:
!ls

credit_card.ipynb submission.csv    test_data.csv     train_data.csv


In [54]:
train_data.head()

,id,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,y
0,0,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,1,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,2,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,3,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
4,4,50000,1,1,2,37,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0


In [55]:
train_data.describe()

,id,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,y
count,27000.00000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,...,27000.000000,27000.000000,27000.000000,27000.000000,2.700000e+04,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000
mean,13499.50000,166856.284444,1.603074,1.853741,1.551556,35.502407,-0.012519,-0.128185,-0.162370,-0.216037,...,43247.852074,40345.600926,38894.870926,5644.593222,5.934313e+03,5229.052593,4855.124630,4782.051667,5238.613148,0.222000
std,7794.37297,129363.239705,0.489269,0.789234,0.521767,9.221539,1.121953,1.196953,1.195634,1.169587,...,64150.721252,60681.919736,59547.938323,16499.811285,2.360826e+04,17745.539585,15826.973077,15154.386855,17865.467132,0.415599
min,0.00000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6749.75000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2392.750000,1798.500000,1287.000000,1000.000000,8.500000e+02,396.000000,300.000000,272.000000,140.000000,0.000000
50%,13499.50000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,...,19114.000000,18219.500000,17179.000000,2100.000000,2.009000e+03,1800.000000,1500.000000,1500.000000,1500.000000,0.000000
75%,20249.25000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,...,54537.750000,50211.000000,49243.500000,5007.000000,5.000000e+03,4500.000000,4014.250000,4069.000000,4002.250000,0.000000
max,26999.00000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.000000,621000.000000,426529.000000,528666.000000,1.000000
